# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#creating dataframe from part one
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

#accidentally saved the csv with the index - removed extra column
del cities_df['Unnamed: 0']

#display dataframe
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saskylakh,71.92,114.08,1.17,92,90,6.31,RU,1603730732
1,Kismayo,-0.36,42.55,80.29,80,55,13.47,SO,1603730683
2,Bluff,-46.60,168.33,44.19,85,20,2.98,NZ,1603730734
3,Mataura,-46.19,168.86,36.21,94,28,1.90,NZ,1603730735
4,East London,-33.02,27.91,62.60,93,90,16.11,ZA,1603730737


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#lat and lng as locations
locations = cities_df[["Lat", "Lng"]]

#humidity as weight
weights = cities_df['Humidity']

In [4]:
# humidity Heatmap layer
fig = gmaps.figure(zoom_level=2, center=(0,30))

#paramaters for heatmap
humidity_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#adding the layer to the map
fig.add_layer(humidity_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#creating the new dataframe with narrowed down cities to meet weather conditions
weather_df = cities_df.loc[(cities_df['Max Temp'] <= 90) & (cities_df['Max Temp'] >= 65)
                           & (cities_df['Cloudiness'] == 0) & (cities_df['Wind Speed'] >= 11)].dropna(how='any')
#display dataframe
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Mar del Plata,-38.00,-57.56,66.20,34,0,13.87,AR,1603730809
81,Aswān,24.09,32.91,84.78,26,0,11.63,EG,1603730831
148,Dolores,-36.31,-57.68,67.21,47,0,11.83,AR,1603730915
196,Salamiyah,35.01,37.05,74.93,19,0,13.02,SY,1603730977
215,Carnarvon,-24.87,113.63,69.80,88,0,19.46,AU,1603730999
296,Francistown,-21.17,27.51,86.00,16,0,11.41,BW,1603731098
362,Assiut,27.18,31.18,82.40,45,0,11.41,EG,1603731179
409,Bahía Blanca,-38.72,-62.27,77.25,18,0,11.30,AR,1603730976
417,Lagoa,39.05,-27.98,67.30,87,0,24.63,PT,1603731248


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#storing the weather dataframe into hotel_df variable
hotel_df = weather_df.loc[:,['City','Country','Lat','Lng']]
hotel_df

#adding empty column
hotel_df['Hotel Name'] = ""

#display dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Mar del Plata,AR,-38.00,-57.56,
81,Aswān,EG,24.09,32.91,
148,Dolores,AR,-36.31,-57.68,
196,Salamiyah,SY,35.01,37.05,
215,Carnarvon,AU,-24.87,113.63,
296,Francistown,BW,-21.17,27.51,
362,Assiut,EG,27.18,31.18,
409,Bahía Blanca,AR,-38.72,-62.27,
417,Lagoa,PT,39.05,-27.98,


In [7]:
#params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# use iterrows to iterate through hotels dataframe
for index, row in hotel_df.iterrows():

    #params for location
    lat = row['Lat']
    lng = row['Lng']
    cities = row['City']

    params["location"] = f"{lat},{lng}"
    
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cities}.")
    
    #base_url for google api
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #params['keyword'] = 'hotel'
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {cities} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    time.sleep(1)
    
    print("------------")

Retrieving Results for Index 63: Mar del Plata.
Closest hotel in Mar del Plata is Gran Hotel Mar del Plata habitaciones.
------------
Retrieving Results for Index 81: Aswān.
Closest hotel in Aswān is Nuba Nile.
------------
Retrieving Results for Index 148: Dolores.
Closest hotel in Dolores is Apart Hotel - Arianos.
------------
Retrieving Results for Index 196: Salamiyah.
Closest hotel in Salamiyah is Gondolas Forum grandchildren and grandparents.
------------
Retrieving Results for Index 215: Carnarvon.
Closest hotel in Carnarvon is Hospitality Carnarvon.
------------
Retrieving Results for Index 296: Francistown.
Closest hotel in Francistown is Budget House.
------------
Retrieving Results for Index 362: Assiut.
Closest hotel in Assiut is Florence Hotel Assiut.
------------
Retrieving Results for Index 409: Bahía Blanca.
Closest hotel in Bahía Blanca is Hotel Argos.
------------
Retrieving Results for Index 417: Lagoa.
Closest hotel in Lagoa is Casa Das Faias.
------------


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
#display dataframe with hotel name column filled
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Mar del Plata,AR,-38.00,-57.56,Gran Hotel Mar del Plata habitaciones
81,Aswān,EG,24.09,32.91,Nuba Nile
148,Dolores,AR,-36.31,-57.68,Apart Hotel - Arianos
196,Salamiyah,SY,35.01,37.05,Gondolas Forum grandchildren and grandparents
215,Carnarvon,AU,-24.87,113.63,Hospitality Carnarvon
296,Francistown,BW,-21.17,27.51,Budget House
362,Assiut,EG,27.18,31.18,Florence Hotel Assiut
409,Bahía Blanca,AR,-38.72,-62.27,Hotel Argos
417,Lagoa,PT,39.05,-27.98,Casa Das Faias


In [24]:
#assign the marker layer to a variable
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add the layer to the map
fig.add_layer(marker_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))